# SCD Type 2 #2 run to initiate next delta loads data

# Generate Fake Data using [faker](https://github.com/joke2k/faker)

### Faker is a Python package that generates fake data for you. Whether you need to bootstrap your database, create good-looking XML documents, fill-in your persistence to stress test it, or anonymize data taken from a production service, Faker is for you.

In [1]:
from __future__ import print_function

import os
import sys

In [2]:
import warnings

warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", "DeprecationWarning")
warnings.filterwarnings("ignore", "SAWarning")

In [3]:
import collections
import random
from collections import *

import numpy as np
import pandas as pd
import csv
import pandavro as pdx

In [4]:
import datetime
import time
import unicodedata
from datetime import *

from dateutil.relativedelta import relativedelta

if sys.version_info[0] >= 3:
    unicode = str

In [5]:
from faker import Faker
from faker_web import WebProvider

In [6]:
import tqdm
from tqdm import *

In [7]:
pd.set_option("display.max_columns", 512)
pd.set_option("display.max_rows", 512)
pd.set_option("display.width", 1024)
pd.set_option("display.max_info_rows", 512)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("mode.chained_assignment", "warn")
pd.set_option("float_format", "{:6.2f}".format)
pd.set_option("display.notebook_repr_html", True)

## Instantiate a faker object generator

In [8]:
fake = Faker()
fake.add_provider(WebProvider)
Faker.seed(0)

## Restore business keys previously used

In [9]:
import pickle

ssns = pickle.load(open("customer_ssn.p", "rb"))

### Set the base path that will be used to persists the data files

In [10]:
destBaseDataPath = "/media/psf/Downloads/performance/data"

## Function to generate fake customer data

In [11]:
import random


def data_gen(count, seed, env):
    fakeDataDico = []
    fakeDataTuple = []
    fakeColumns = OrderedDict()

    Faker.seed(seed)
    tday = datetime.now().date()

    m = datetime.now()
    n = datetime(m.year, m.month, m.day, m.hour, m.minute, m.second)
    e = datetime(2099, 12, 31, 0, 0, 0)

    for _ in tnrange(count, desc="Data generator", leave=True):
        ssn = fake.ssn()

        if ssn in ssns:
            fakeDataDico.append(
                OrderedDict(
                    {
                        "customer_ssn": ssn,
                        "customer_job": None,
                        "customer_job_rank": random.randint(1, 100),
                        "start_date": n,
                        "end_date": e,
                        "is_current": True,
                    }
                )
            )
        else:
            fakeDataDico.append(
                OrderedDict(
                    {
                        "customer_ssn": ssn,
                        "customer_job": fake.job(),
                        "customer_job_rank": 0,
                        "start_date": n,
                        "end_date": e,
                        "is_current": True,
                    }
                )
            )
            ssns.append(ssn)

    df_0 = pd.DataFrame(data=fakeDataDico)
    df = df_0[sorted(df_0.columns)].copy(deep=True)

    # -------------------------------------------------------------------------------
    # -------------------------------------------------------------------------------
    # -------------------------------------------------------------------------------
    tmpPath = f"{destBaseDataPath}/generator_scd_{env}"

    try:
        os.makedirs(tmpPath)
    except:
        print("Dir already exists")
        pass
    
    try:
        sys.path.remove("{1}/fake_customers_{0:>02}.avro".format(seed, tmpPath))
    except:
        print("old file has been removed")
        pass
      

    pdx.to_avro(
        "{1}/_tmp_fake_customers_{0:>02}.avro".format(seed, tmpPath),
        df,
    )

    os.rename(
        "{1}/_tmp_fake_customers_{0:>02}.avro".format(seed, tmpPath),
        "{1}/fake_customers_{0:>02}.avro".format(seed, tmpPath),
    )

    return df

# Loop to generate fake customers files

- set how many files will be created
- set how many row per file will be generated
- set an environment lile "prd",...

In [12]:
max_file = 1
max_row = 15
env = "prd"

In [13]:
dfx = None
for s in tnrange(max_file, desc="File generator"):
    dfx = data_gen(max_row, s, env)

File generator:   0%|          | 0/1 [00:00<?, ?it/s]

Data generator:   0%|          | 0/15 [00:00<?, ?it/s]

Dir already exists
old file has been removed


### persists the generated business keys

In [14]:
import pickle

pickle.dump(list(set(dfx.customer_ssn.to_list())), open("customer_ssn.p", "wb"))